In [1]:
!pip install langchain_community[all]
!pip install llama-parse
!pip install qdrant_client
!pip install langchain_qdrant
!pip install ollama

Import Library



In [2]:
import os
import uuid
from langchain_community.document_loaders import PyPDFLoader, TextLoader, UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from qdrant_client import QdrantClient, models
from langchain_qdrant import FastEmbedSparse, Qdrant
from langchain_community.vectorstores import Qdrant
from langchain_core.documents import Document
import nest_asyncio
from llama_parse import LlamaParse
import ollama
from langchain_community.embeddings import OllamaEmbeddings

c:\Users\ASUS-3070\Desktop\Github\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
nest_asyncio.apply()

API

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
qdrant_url = os.environ["QDRANT_URL"]
qdrant_api_key = os.environ["QDRANT_API_KEY"]
collection_name = "Ollama-RAG"

qdrant_client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

In [6]:
# --- Vectorstore Initialization (Langchain Qdrant) ---
vectorstore = Qdrant(
    client=qdrant_client,
    collection_name=collection_name,
    embeddings=OllamaEmbeddings,  # We're handling embeddings separately
)

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\3637584716.py:2: LangChainDeprecationWarning: The class `Qdrant` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-qdrant package and should be used instead. To use it run `pip install -U :class:`~langchain-qdrant` and import as `from :class:`~langchain_qdrant import Qdrant``.
  vectorstore = Qdrant(
c:\Users\ASUS-3070\Desktop\Github\.venv\lib\site-packages\langchain_community\vectorstores\qdrant.py:152: UserWarning: `embeddings` should be an instance of `Embeddings`.Using `embeddings` as `embedding_function` which is deprecated
  warnings.warn(


Hybrid Search Function

In [7]:
def hybrid_search(query: str, collection: str, limit: int = 5):
    dense_vector = ollama.embeddings(model='rjmalagon/gte-qwen2-1.5b-instruct-embed-f16', prompt=query)

    results = qdrant_client.search(
            collection_name=collection_name,
            query_vector=models.NamedVector(
                name="gte-qwen1.5",
                vector=dense_vector['embedding']
            ),
            limit=limit,
            search_params=models.SearchParams(
                hnsw_ef=128
            )
    )
    return results

In [8]:
hybrid_search("What is machine learning?", "Ollama-RAG")

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


[ScoredPoint(id=8, version=0, score=0.62523055, payload={'text': '1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.', 'metadata': {'source': 'C:/Users/shaoe/OneDrive/Desktop/AAI3008LLM/The_Hundred_page_Machine_Learning_Book_Andriy_Burkov_Z_Library.pdf', 'page': 2}}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=4, version=0, score=0.5865381, payload={'text': 'As you can see, just like artiﬁcial intelligence is not intelligence, machine learning is not\nlearning. However, machine learning is a universally recognized term that usu

In [9]:
def retrieve_documents(query):
    results = hybrid_search(query, collection=collection_name, limit=3)  # Use hybrid search here
    list = "Database Context: "
    for i in range(len(results)):
        list += str(results[i].payload['text'])
    return list

retrieve_documents("What is machine learning?")

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(


'Database Context: 1 Introduction\n1.1 What is Machine Learning\nMachine learning is a subﬁeld of computer science that is concerned with building algorithms\nwhich, to be useful, rely on a collection of examples of some phenomenon. These examples\ncan come from nature, be handcrafted by humans or generated by another algorithm.\nMachine learning can also be deﬁned as the process of solving a practical problem by 1)\ngathering a dataset, and 2) algorithmically building a statistical model based on that dataset.As you can see, just like artiﬁcial intelligence is not intelligence, machine learning is not\nlearning. However, machine learning is a universally recognized term that usually refers\nto the science and engineering of building machines capable of doing various useful things\nwithout being explicitly programmed to do so. So, the word “learning” in the term is used\nby analogy with the learning in animals rather than literally.\nWho This Book is ForPreface\nLet’s start by telling 

RAG Chain

In [10]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities import GoogleSearchAPIWrapper

In [11]:
def get_rag_chain(enable_search=False):
    """Creates and returns the RAG chain with hybrid search and optional Google Search."""

    # Initialize the LLM
    llm = Ollama(model="phi3:mini")

    # Create a prompt template
    template = """Answer the question based on the following context:
    {context}
    {online_context}

    Question: {question}
    """
    prompt = ChatPromptTemplate.from_template(template)

    # Function to retrieve relevant documents using hybrid search
    def retrieve_documents(query):
        results = hybrid_search(query, collection=collection_name, limit=3)  # Upgrade to hybrid search here
        list = "Database Context: "
        for i in range(len(results)):
            list += str(results[i].payload['text'])
        #print(f"{list}") # This one maybe fix to add metadata content
        return list 

    # Add online searching, defaulting to empty string if search is disabled
    def online_search(query):
        """Searches Google and returns a summary of the results."""
        search = GoogleSearchAPIWrapper()
        search_results = search.run(query)
        if not search_results:
            search_results = "No relevant information found online."  # Handle empty results
        #print(f"Online Search Results: {search_results}")  # Debugging
        return f"Online search results: {search_results}"
    
    def filler(query):
        #print("Online Search Switched off")
        return f"Online Search Switched Off"
    
    if enable_search:
        rag_chain = (
        {"context": retrieve_documents, ### This need 
         "question": RunnablePassthrough(),
         "online_context": online_search} ### This is correct
        | prompt
        | llm
        | StrOutputParser()
    )
    else:
        rag_chain = (
        {"context": retrieve_documents, ### This need 
         "question": RunnablePassthrough(),
         "online_context": filler} ### This is correct
        | prompt
        | llm
        | StrOutputParser()
    )

    return rag_chain

Search Tool

In [12]:
def agent_search(query):
    """Search for relevant information in the document and online."""
    rag_chain = get_rag_chain(enable_search=False)  # Enable online search
    response = rag_chain.invoke(query)
    return response

In [13]:
from langchain.agents import Tool

In [14]:
tools = [
    Tool(
        name="document_search",
        func=agent_search,
        description="useful for when you need to answer questions about the document and online."
    )
]

In [15]:
# --- Agent Initialization ---
from langchain_community.llms import Ollama
from langchain.agents import initialize_agent
from langchain.agents import AgentType

In [16]:
llm = Ollama(model="phi3:mini")
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\1301706811.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi3:mini")
C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\1301706811.py:2: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://pytho

In [17]:
from langchain_google_community import GoogleSearchAPIWrapper

In [ ]:
## WAKE UP
try:
    agent.run("Hello")
except:
    pass

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\2047023263.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent.run("Hello")




> Entering new AgentExecutor chain...
Question: How do you say "Hello" in French?
Thought: To find out how to greet someone in French, I can use a search engine.
Action: document_search
Action Input: "How do you say hello in French?"

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(



Observation: Bonjour!
Thought:Question: How do you say "Hello" in French?
Thought: To find out how to greet someone in French, I can use a search engine.
Action: document_search
Action Input: "How do you say hello in French?"

C:\Users\ASUS-3070\AppData\Local\Temp\ipykernel_16196\185713141.py:4: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = qdrant_client.search(



Observation: In the provided context, there is no mention of saying "hello" or greetings in French. Therefore, based on this specific passage alone, I cannot provide a direct translation for how to say hello in French from the given text. However, generally speaking, you would say "Bonjour" as an equivalent way to say hello in French that matches your query best.
Thought:Question: How do you say "Hello" in French?
Thought: To find out how to greet someone in French, I can use a search engine because the document provided does not contain this information directly within its text. Since documents_search seems unreliable for general knowledge questions like these, an alternative online resource is needed.
Action: Seeking help from another source by rephrasing my question to fit available tools or contexts where translations are typically found (e.g., dictionaries). However, as the instruction does not provide a direct way of accessing such resources and since I am only allowed 'document

In [19]:
import warnings
warnings.filterwarnings('ignore')

query = "What is machine learning?"
response = agent.run(query)
print(response)



> Entering new AgentExecutor chain...
Question: What is machine learning?
Thought: Machine learning involves algorithms and statistical models that enable computers to perform tasks without explicit instructions, relying on patterns and inference instead. I need to find a reliable source for this definition.
Action: document_search
Action Input: "What is machine learning?"
Observation: Machine Learning, which falls under computer science and engineering disciplines, involves creating algorithms that find mathematical formulas to solve practical problems based on a collection of examples or dataset without being explicitly programmed for the specific task. The term "learning" in Machine Learning refers loosely by analogy with how animals learn through experience rather than literal learning ability.
Thought:Question: What is machine learning?
Thought: To provide a comprehensive definition, I'll consult reliable sources on computer science and engineering disciplines that define this t